In [ ]:
# aligns atlasnet models by rotating it about the y axis by 90 degrees, normalizes the meshes, and saves them as an .obj 

In [1]:
%load_ext autoreload
%autoreload 2
notebook_fixed_dir = False

In [2]:
# this cell can only be called once
import os
if not notebook_fixed_dir:
    os.chdir('..')
    notebook_fixed_dir = True
print(os.getcwd())

/home/svcl-oowl/brandon/research/sil_consistent_at_inference


In [18]:
import pprint
import pickle
import glob
import random
from pathlib import Path
import os

import torch
from tqdm.autonotebook import tqdm
from PIL import Image
import numpy as np
from pytorch3d.renderer import look_at_view_transform
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import pytorch3d
from pytorch3d.io import load_obj
from pytorch3d.io import save_obj
from pytorch3d.renderer.cameras import look_at_view_transform
from pytorch3d.renderer import Textures
import trimesh

from utils import utils

In [24]:
# points: a tensor of 3d points
def normalize_pointclouds_numpy(points):
    max_vert_values = np.amax(points, axis=0)
    min_vert_values = np.amin(points, axis=0)
    max_width = np.amax(max_vert_values-min_vert_values)
    normalized_points = points * (1/max_width)

    return normalized_points

def rot_y(theta, degrees=True):
    if degrees:
        theta = theta * (np.pi/180)
    rot_matrix = np.array([[np.cos(theta), 0, np.sin(theta)],
                           [0,1,0],
                           [-np.sin(theta), 0, np.cos(theta)]
                          ])
    return rot_matrix

# based on https://github.com/facebookresearch/pytorch3d/issues/51
def load_untextured_mesh(mesh_path, device):
    mesh = load_objs_as_meshes([mesh_path], device=device, load_textures = False)
    verts, faces_idx, _ = load_obj(mesh_path)
    faces = faces_idx.verts_idx
    verts_rgb = torch.ones_like(verts)[None] # (1, V, 3)
    textures = Textures(verts_rgb=verts_rgb.to(device))
    mesh_no_texture = Meshes(
        verts=[verts.to(device)],
        faces=[faces.to(device)],
        textures=textures
        )
    return mesh_no_texture


In [32]:
class_ids = ["02691156", "02828884", "02933112", "02958343", "03001627", "03211117", "03636649", "03691459", "04090263", "04256520", "04379243", "04401088", "04530566"]
#class_ids=["test_recs"]

rec_dir = "/home/svcl-oowl/brandon/research/AtlasNet/data/pytorch_3d_render_recs"

In [33]:
for class_name in class_ids:
    meshes_dir = os.path.join(rec_dir, class_name)
    mesh_paths = [str(path) for path in Path(meshes_dir).rglob("*.ply")]
    print(class_name)
    for mesh_path in tqdm(mesh_paths):
        mesh = trimesh.load_mesh(mesh_path, process=False, force="mesh")
        mesh.vertices = normalize_pointclouds_numpy(mesh.vertices @ rot_y(90))
        output_mesh_path = mesh_path.replace(".ply", ".obj")
        mesh.export(output_mesh_path)
            

02691156



02828884



02933112



02958343



03001627


...


03636649



03691459



04090263



04401088



04530566
